# Query bucket logs using BigQuery

This notebook walks you through how to use bigquery to analyze our usage logging using bigquery.

We are using bigquery for several reasons:
1. **Saves us time:** Uploading logs from a bucket to bigquery is _fast_, waaaay faster than trying to `gcloud storage rsync` files to a VM or to your laptop.
2. **Existing tooling:** Google has some existing tooling and data analysis products built around bigquery that we can use if we want to

NOTE: If you have not yet logged in to Google Cloud using the `gcloud` command, be sure to run the following before running the other cells:

```bash
gcloud auth login --project=sdss-natcap-gef-ckan
```

In [ ]:
import os
import json
import datetime

from datetime import date
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

client = bigquery.Client()
tablename = f"sdss-natcap-gef-ckan.data_cache_logs.natcap-data-cache-analysis-{date.today().strftime("%Y-%m-%d")}"
print(f"Considering the table {tablename}")

####### Set these to change the dates of the analysis
QUERY_START_DATE = datetime.datetime(year=2025, month=6, day=1, hour=0, minute=0, second=0)
QUERY_END_DATE = datetime.datetime(year=2025, month=6, day=30, hour=23, minute=59, second=59)
print(f"Queries are in the date range {QUERY_START_DATE} - {QUERY_END_DATE}")

# Convert from seconds to microseconds
# Bucket timestamps are in microseconds, but python's .timestamp() is seconds.
START_MICROS = QUERY_START_DATE.timestamp() * 1000000
END_MICROS = QUERY_END_DATE.timestamp() * 1000000

# A convenience function to query bigquery and convert the output to a dataframe.

def _query_to_dataframe(query):
    return client.query_and_wait(query).to_dataframe()

In [ ]:
try:
    table = client.get_table(tablename)
except (NotFound, ValueError):
    with open('cloud_storage_usage_schema_v0.json') as schema_file:
        schema = json.load(schema_file)
    table = bigquery.Table(tablename, schema=schema)
    table = client.create_table(table)  # this actually makes the API request
    print(f"Created table {tablename}")

if table.num_rows > 0:
    print("Table already has some stuff in it.")
    print(f"If this is incorrect, please delete the table {tablename} and re-run this cell.")
else:
    print("Loading table from GCS")
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        skip_leading_rows=1,
        source_format=bigquery.SourceFormat.CSV
    )

    client.load_table_from_uri(
        "gs://natcap-data-access-logs/natcap-data-cache/natcap-data-cache_usage_*",
        tablename, job_config=job_config
    )
    print("Finished loading from usage CSVs, reloading table")
    table = bigquery.Table(tablename, schema=schema)
    print("Table finished loading")

In [ ]:
# Top 25 downloads by data volume within the analysis period
query = f"""
SELECT cs_object, SUM(sc_bytes)/1e9 as gigabytes, count(cs_object) as http_requests
  FROM `{tablename}`
  where 
      sc_status >= 200 and
      sc_status < 300 and
      time_micros >= {START_MICROS} and
      time_micros <= {END_MICROS}
  group by cs_object 
  order by gigabytes desc 
  LIMIT 25 
"""
_query_to_dataframe(query)

In [ ]:
# Total egress for all data in the table within the time period
query = f"""
SELECT SUM(sc_bytes)/1e12 as terabytes
  FROM `{tablename}`
  where 
      sc_status >= 200 and
      sc_status < 300 and
      time_micros >= {START_MICROS} and
      time_micros <= {END_MICROS}
"""
_query_to_dataframe(query)

In [ ]:
# Get top 25 files that have been downloaded directly from the data hub within the analysis period
#
# The cs_referer attribute has the page that the link came from.
referred_from_datahub_query = f"""
SELECT cs_object, cs_referer, count(cs_object) as http_requests
    FROM
        (SELECT REGEXP_REPLACE(cs_object, "^'", "") as cs_object, cs_referer, sc_status, cs_method FROM
            `{tablename}`)
    WHERE sc_status >= 200 
        AND sc_status < 300
        AND cs_referer LIKE 'https://data.naturalcapitalproject.stanford.edu/dataset/sts-%'
        AND cs_object != ''
        AND cs_method LIKE 'GET'
        AND time_micros >= {START_MICROS}
        AND time_micros <= {END_MICROS}
    GROUP BY cs_object, cs_referer
    ORDER BY http_requests DESC
    LIMIT 25
"""
_query_to_dataframe(referred_from_datahub_query)

In [ ]:
# Get top 25 files that have been downloaded directly from cloud storage within the analysis period
#
# The cs_referer attribute has the page that the link came from, which is empty when the file is directly accessed
# NOTE: the http_requests column indicates how many "chunks" of the object have been downloaded, where
#       a single download of the file would consist of a bunch of chunks.
directly_accessed_query = f"""
SELECT cs_object, cs_referer, count(cs_referer) as http_requests
    FROM
        (SELECT REGEXP_REPLACE(cs_object, "^'", "") as cs_object, cs_referer, sc_status, cs_method, c_ip FROM
            `{tablename}`)
    WHERE sc_status >= 200 
        AND sc_status < 300
        AND cs_referer IS NOT NULL
        AND cs_referer LIKE ''
        AND cs_object != ''
        AND cs_method LIKE 'HEAD'
        AND time_micros >= {START_MICROS}
        AND time_micros <= {END_MICROS}
    GROUP BY cs_referer, cs_object
    ORDER BY http_requests DESC
    LIMIT 25
"""
_query_to_dataframe(directly_accessed_query)

In [ ]:
# Get the top 25 files by request counts where the requests have come from GDAL within the analysis period

gdal_accesses = f"""
SELECT c_ip, cs_object, cs_user_agent, count(cs_method) as n
    FROM
        (SELECT REGEXP_REPLACE(cs_object, "^'", "") as cs_object, cs_referer, sc_status, cs_user_agent FROM
            `{tablename}`)
    WHERE sc_status >= 200 
        AND sc_status < 300
        AND cs_object != ''
        AND cs_user_agent LIKE "GDAL/%"
        AND time_micros >= {START_MICROS}
        AND time_micros <= {END_MICROS}
    group by c_ip, cs_object, cs_user_agent, cs_method
    order by n desc
    LIMIT 25

"""

_query_to_dataframe(gdal_accesses)

In [ ]:
# Get all user agents listed in the logs during the analysis period
user_agents = f"""
SELECT cs_user_agent, count(cs_user_agent) as n
    FROM `{tablename}`
    WHERE sc_status >= 200
        AND sc_status < 300
        AND cs_object != ''
        AND time_micros >= {START_MICROS}
        AND time_micros <= {END_MICROS}
    GROUP BY cs_user_agent
    ORDER BY n DESC
"""
_query_to_dataframe(user_agents)

In [ ]:
# Get all browser user agents listed in the logs during the analysis period
user_agents = f"""
SELECT cs_user_agent, count(cs_user_agent) as n
    FROM `{tablename}`
    WHERE sc_status >= 200
        AND sc_status < 300
        AND cs_object != ''
        AND time_micros >= {START_MICROS}
        AND time_micros <= {END_MICROS}
        AND cs_user_agent LIKE "Mozilla/%"
    GROUP BY cs_user_agent
    ORDER BY n DESC
"""
_query_to_dataframe(user_agents)

In [ ]:
# Get all non-browser user agents listed in the logs during the analysis period
user_agents = f"""
SELECT cs_user_agent, count(cs_user_agent) as n
    FROM `{tablename}`
    WHERE sc_status >= 200
        AND sc_status < 300
        AND cs_object != ''
        AND time_micros >= {START_MICROS}
        AND time_micros <= {END_MICROS}
        AND cs_user_agent NOT LIKE "Mozilla/%"
    GROUP BY cs_user_agent
    ORDER BY n DESC
"""
_query_to_dataframe(user_agents)

# After you're done

Once you've finished working with BigQuery data, be sure to delete the table you created!
Bigquery doesn't have a way to easily delete rows, and tables are generally append-only, so deleting the table is the only way.

Example (but change the table name to your table name):

```shell
bq rm data_cache_logs.natcap-data-cache-analysis-2025-06-04
```
